### Data
Downloaded from: https://www.kaggle.com/unsdsn/world-happiness

### Question 1.

Summary of data set: The data set is a five-year survey of happiness of about 150 contries and some figures such as GDP or health level which are expected to have any correlation to happiness. Given data set includes 5 files in csv format named from 2015.csv to 2019.csv.

Suppose that we want to find countries whose happiness scores does not decrease through those 5 years (i.e., equal to or greater than that of previous years).

### 문제 1.

데이터셋 요약: 5년간에 걸쳐 약 150개 국가에서 조사한 행복지수와 행복에 관련이 있을 것으로 예상되는 GDP나 건강과 같은 지수를 담고 있다. 2015년부터 2019년까지 조사결과가 각각 2015.csv ~ 2019.csv 파일에 들어있다. 매해 조사 국가가 조금씩 차이가 있으며 컬럼 가지수나 이름 또한 조금씩 다르다. 따라서 데이터를 분석하기 위해 컬럼이름을 직접 출력해보고 확인하여야 한다.

첫번째 문제는 5년동안 행복지수가 지속적으로 상승하거나 적어도 전년도와 같은 나라를 찾는 것이다.

In [2]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my app").master("local").getOrCreate()

# get context from the session
sc = spark.sparkContext

22/11/25 10:45:08 WARN Utils: Your hostname, bagdoyeong-ui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 192.168.0.13 instead (on interface en0)
22/11/25 10:45:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/25 10:45:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

(1) 2015.csv ~ 2019.csv의 파일 4개를 각각 data2015 ~ data2019 란 이름의 데이터프레임으로 읽어들이시오. 그리고 각 데이터프레임의 schema를 출력하시오. (10점)

In [4]:
data2015 = spark.read.format('csv').option("sep", ",").option('header', 'true').load('./Dataset/World Hapiness Report/2015.csv')
data2016 = spark.read.format('csv').option("sep", ",").option('header', 'true').load('./Dataset/World Hapiness Report/2016.csv')
data2017 = spark.read.format('csv').option("sep", ",").option('header', 'true').load('./Dataset/World Hapiness Report/2017.csv')
data2018 = spark.read.format('csv').option("sep", ",").option('header', 'true').load('./Dataset/World Hapiness Report/2018.csv')
data2019 = spark.read.format('csv').option("sep", ",").option('header', 'true').load('./Dataset/World Hapiness Report/2019.csv')

In [29]:
data2018.show(5)

+------------+-----------------+-----+--------------+--------------+-----------------------+----------------------------+----------+-------------------------+
|Overall rank|Country or region|Score|GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption|
+------------+-----------------+-----+--------------+--------------+-----------------------+----------------------------+----------+-------------------------+
|           1|          Finland|7.632|         1.305|         1.592|                  0.874|                       0.681|     0.202|                    0.393|
|           2|           Norway|7.594|         1.456|         1.582|                  0.861|                       0.686|     0.286|                    0.340|
|           3|          Denmark|7.555|         1.351|         1.590|                  0.868|                       0.683|     0.284|                    0.408|
|           4|          Iceland|7.495|        

In [30]:
data2019.columns

['Overall rank',
 'Country or region',
 'Score',
 'GDP per capita',
 'Social support',
 'Healthy life expectancy',
 'Freedom to make life choices',
 'Generosity',
 'Perceptions of corruption']

In [5]:
# column : 나라-str•행복지수-float•인구당GDP-float•건강float•자유도-float•관용도-float

data2015 = data2015.select('Country', 'Happiness Score', 'Economy (GDP per Capita)', 'Health (Life Expectancy)', 'Freedom', 'Generosity')
data2016 = data2016.select('Country', 'Happiness Score', 'Economy (GDP per Capita)', 'Health (Life Expectancy)', 'Freedom', 'Generosity')
data2017 = data2017.select('Country', '`Happiness.Score`', '`Economy..GDP.per.Capita.`', '`Health..Life.Expectancy.`', 'Freedom', 'Generosity')
data2018 = data2018.select('Country or region', 'Score', 'GDP per Capita', 'Healthy Life Expectancy', 'Freedom to make life choices', 'Generosity')
data2019 = data2019.select('Country or region', 'Score', 'GDP per Capita', 'Healthy Life Expectancy', 'Freedom to make life choices', 'Generosity')

In [6]:
data_col = data2015.columns[1:]

for c in data_col:
    data2015 = data2015.withColumn(c, data2015[c].cast(FloatType()))

In [7]:
data_col1 = data2016.columns[1:]
data_col2 = data2017.columns[1:]
data_col3 = data2018.columns[1:]
data_col4 = data2019.columns[1:]

for c in data_col1:
    data2016 = data2016.withColumn(c, data2016[c].cast(FloatType()))
    
for c in data_col3:
    data2018 = data2018.withColumn(c, data2018[c].cast(FloatType()))
    

for c in data_col4:
    data2019 = data2019.withColumn(c, data2019[c].cast(FloatType()))

In [8]:
data_col2 = ['`Happiness.Score`',
     '`Economy..GDP.per.Capita.`',
     '`Health..Life.Expectancy.`',
     'Freedom',
     'Generosity']

for c in data_col2:
    data2017 = data2017.withColumn(c, data2017[c].cast(FloatType()))

In [9]:
data2015.printSchema()
data2016.printSchema()
data2017.printSchema()
data2018.printSchema()
data2019.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Happiness Score: float (nullable = true)
 |-- Economy (GDP per Capita): float (nullable = true)
 |-- Health (Life Expectancy): float (nullable = true)
 |-- Freedom: float (nullable = true)
 |-- Generosity: float (nullable = true)

root
 |-- Country: string (nullable = true)
 |-- Happiness Score: float (nullable = true)
 |-- Economy (GDP per Capita): float (nullable = true)
 |-- Health (Life Expectancy): float (nullable = true)
 |-- Freedom: float (nullable = true)
 |-- Generosity: float (nullable = true)

root
 |-- Country: string (nullable = true)
 |-- Happiness.Score: string (nullable = true)
 |-- Economy..GDP.per.Capita.: string (nullable = true)
 |-- Health..Life.Expectancy.: string (nullable = true)
 |-- Freedom: float (nullable = true)
 |-- Generosity: float (nullable = true)
 |-- `Happiness.Score`: float (nullable = true)
 |-- `Economy..GDP.per.Capita.`: float (nullable = true)
 |-- `Health..Life.Expectancy.`: float (nullable = tr

(2) 5년간의 데이터프레임에서 나라와 행복지수를 뽑아 키-값의 쌍 (즉, (나라, 행복지수))의 RDD로 생성하시오. 그리고 5개의 RDD를 rdd15 ~ rdd19이란 이름 변수에 저장하시오. 위에서 말했듯이 나라이름이 Country나 Country or region와 같은 이름의 컬럼으로 들어가 있으므로 확인하고 처리해야 할 것이다. (10점)

힌트: 혹시 dataframe select를 하려면 Happiness.Score와 같이 column name에 점(.)이 있으면 Happiness를 table name으로 인식하기 때문에 이를 escape해주어야 한다. select시 column name을 "\`Happiness.Score\`" 와 같이 reverse single quotation mark (\`)로 감싸주면 된다. 하지만 dataframe select를 하지 않는 방법도 있고 코드도 더 간단하다.

In [10]:
rdd15 = data2015.select('Country', 'Happiness Score')
rdd16 = data2016.select(col('Country').alias('Country2'), col('Happiness Score').alias('Happiness Score2'))
rdd17 = data2017.select(col('Country').alias('Country3'), col('`Happiness.Score`').alias('Happiness Score3'))
rdd18 = data2018.select(col('Country or region').alias('Country4'), col('Score').alias('Happiness Score4'))
rdd19 = data2019.select(col('Country or region').alias('Country5'), col('Score').alias('Happiness Score5'))

In [11]:
rdd19.show(5)

+-----------+----------------+
|   Country5|Happiness Score5|
+-----------+----------------+
|    Finland|           7.769|
|    Denmark|             7.6|
|     Norway|           7.554|
|    Iceland|           7.494|
|Netherlands|           7.488|
+-----------+----------------+
only showing top 5 rows



(3) cogroup 이용해 연속된 두 해의 RDD, rdd15와 rdd16에서 같은 키 (즉, 나라)를 갖는 두 해의 행복지수를 묶어 2015 년도에 비해 2016 년도에 행복지수가 증가하거나 적어도 같은 나라를 filtering할 수 있다. 이러한 나라에 대해 (나라, 2016년도의 행복지수)의 키-값의 쌍 RDD를 생성해 rdd란 이름으로 저장하시오. (15점) 

주의: 해마다 조사국가가 조금씩 차이가 있으므로 5년간 모두 행복지수 값이 존재하는 국가만을 대상으로 한다. 즉, cogroup시 행복지수가 없으면 filter out시켜야 할 것이다.

힌트: cogroup을 한 뒤 행복지수가 없는 나라를 판단해 (즉, 길이 > 0) filter한 뒤 각 나라마다 두 해의 행복지수를 비교해 filter를 한뒤 다시 map을 이용해 (나라, 2016년 행복지수)의 키-값 쌍 RDD를 생성한다. 값으로 2016년 행복지수를 놓는 이유는 다음 문제(4)에서 다시 rdd와 그 다음해에 대해 재귀적으로 연산을 처리하기 위함이다.

In [12]:
joinExpression = (rdd15['Country'] == rdd16['Country2']) & (rdd15['Happiness Score'] <= rdd16['Happiness Score2'])
joinType = 'inner'
rdd = rdd15.join(rdd16, joinExpression, joinType)

In [13]:
rdd.show(10)

+--------------+---------------+--------------+----------------+
|       Country|Happiness Score|      Country2|Happiness Score2|
+--------------+---------------+--------------+----------------+
|       Finland|          7.406|       Finland|           7.413|
|   New Zealand|          7.286|   New Zealand|           7.334|
|     Australia|          7.284|     Australia|           7.313|
|       Germany|           6.75|       Germany|           6.994|
|         Chile|           6.67|         Chile|           6.705|
|     Argentina|          6.574|     Argentina|            6.65|
|Czech Republic|          6.505|Czech Republic|           6.596|
|       Uruguay|          6.485|       Uruguay|           6.545|
|      Colombia|          6.477|      Colombia|           6.481|
|      Thailand|          6.455|      Thailand|           6.474|
+--------------+---------------+--------------+----------------+
only showing top 10 rows



(4) 이 rdd를 다음 3개 rdd (즉, rdd17, rdd18, rdd19)와 위와 같은 연산을 연달아 수행하여 최종적으로 5년간 행복지수가 지속적으로 증가하거나 적어도 감소하지 않는 나라를 출력하시오. (15점)

In [14]:
joinExpression = (rdd['Country2'] == rdd17['Country3']) & (rdd['Happiness Score2'] <= rdd17['Happiness Score3'])
joinType = 'inner'
rdd = rdd.join(rdd17, joinExpression, joinType)

In [15]:
joinExpression = (rdd['Country3'] == rdd18['Country4']) & (rdd['Happiness Score3'] <= rdd18['Happiness Score4'])
joinType = 'inner'
rdd = rdd.join(rdd18, joinExpression, joinType)

In [16]:
rdd.count()

37

In [17]:
joinExpression = (rdd['Country4'] == rdd19['Country5']) & (rdd['Happiness Score4'] <= rdd19['Happiness Score5'])
joinType = 'inner'
rdd = rdd.join(rdd19, joinExpression, joinType)

In [18]:
rdd.select('Country').distinct().count()

29

In [19]:
rdd.select('Country').distinct().show(30)

+-------------------+
|            Country|
+-------------------+
|               Chad|
|            Senegal|
|        Philippines|
|           Cambodia|
|        Ivory Coast|
|               Togo|
|           Slovakia|
|            Finland|
|              Benin|
|              Malta|
|         Tajikistan|
|            Burundi|
|              Gabon|
|              Niger|
|            Estonia|
|           Mongolia|
|           Honduras|
|              Syria|
|Congo (Brazzaville)|
|             Latvia|
|     Czech Republic|
| Dominican Republic|
|             Poland|
|           Portugal|
|           Cameroon|
|            Romania|
|             Serbia|
|       Burkina Faso|
|            Hungary|
+-------------------+



### Question 2.

Compute Pearson correlation between happiness score (H) and each of four factors, which are GDP per capita (GDP), health (HL), freedom (FD) and generosity (G). The equation calculating Pearson correlation between two columns is

\begin{equation*}
P_{X, Y} = \frac{1}{n-1} \sum_{i = 1}^n \left( \frac{x_i - \mu_x}{\sigma_x} \right) \left( \frac{y_i - \mu_y}{\sigma_y} \right)
\end{equation*}

where $X = \{ x_1, ..., x_n \}$ and $Y = \{ y_1, ..., y_n \}$ represet sets of two attributes such that $x_i$ and $y_i$ are corresponding columns of the i-th row. Furthermore, $\mu_x$ and $\sigma_x$ denote mean and standard deviation of the corresponding column respectively.

Compute Pearson correlations $P_{H, GDP}$, $P_{H, HL}$, $P_{H, FD}$ and $P_{H, G}$ by filling the following blank cells.

### 문제 2.

5해에 걸친 조사를 통합하여 행복지수(H)와 다른 네가지 요인, 인구당 GDP(GDP) 및 건강(HL), 자유도(FD), 관용도(G) 간의 피어슨 상관계수 (Pearson correlation)을 게산하시오. 계산 식은 다음과 같다.

\begin{equation*}
P_{X, Y} = \frac{1}{n-1} \sum_{i = 1}^n \left( \frac{x_i - \mu_x}{\sigma_x} \right) \left( \frac{y_i - \mu_y}{\sigma_y} \right)
\end{equation*}

단, 위 식에서 $X = \{ x_1, ..., x_n \}$와 $Y = \{ y_1, ..., y_n \}$가 나타내는 것은 두 개의 요인(즉, 칼럼)에 대한 i번째 row의 해당 값을 말한다. 또한 $\mu_x$와 $\sigma_x$는 해당 칼럼의 평균과 표준편차를 의미한다.

아레 셀을 채워 가며 피어슨 상관계수 $P_{H, GDP}$ 및 $P_{H, F}$, $P_{H, HL}$, $P_{H, FD}$, $P_{H, G}$을 계산하시오.

(5) 우선 5개의 데이터프레임 data2015 ~ data2019에서 \[H, GDP, HL, FD, G\]의 numpy arrray로 이루어진 RDD들을 각각 구한 후 모두 union하여 합치시오. 통합한 RDD의 이름은 rdd로 정하시오. 각 데이터프레임 마다 해당 칼럼 이름이 조금씩 다르므로 schema를 보고 적절히 뽑아와야 한다. (10점)

참고: numpy array를 각 elelement로 갖는 RDD를 이용하면 다섯 가지 칼럼의 계산을 한꺼번에 수행할 수 있어 매우 편하다. 예를 들어 numpy array 로 아래와 같은 계산이 가능하다.

Code:
````python
import numpy as np

x = np.array([ 1.0, 2.0, 3.0 ], dtype=np.float16)
y = np.array([ 3.0, 2.0, 1.0 ], dtype=np.float16)

print("x + y = {}".format(x + y))
print("x * y = {}".format(x * y))
````
Output:
````
x + y = [4. 4. 4.]
x * y = [3. 4. 3.]
````

RDD의 element에 4개 칼럼의 값을 numpy array로 넣어두면 .count(), .sum(), .mean(), .stdev() 등의 action을 수행하면 각 칼럼마다의 연산이 한꺼번에 수행된다.

### HINT

lambda x, y : x + y -> x가 vector일 경우?
똑같이 짜도 element wise로 더해짐

In [20]:
data2015.show()

+--------------------+---------------+------------------------+------------------------+-------+----------+
|             Country|Happiness Score|Economy (GDP per Capita)|Health (Life Expectancy)|Freedom|Generosity|
+--------------------+---------------+------------------------+------------------------+-------+----------+
|         Switzerland|          7.587|                 1.39651|                 0.94143|0.66557|   0.29678|
|             Iceland|          7.561|                 1.30232|                 0.94784|0.62877|    0.4363|
|             Denmark|          7.527|                 1.32548|                 0.87464|0.64938|   0.34139|
|              Norway|          7.522|                   1.459|                 0.88521|0.66973|   0.34699|
|              Canada|          7.427|                 1.32629|                 0.90563|0.63297|   0.45811|
|             Finland|          7.406|                 1.29025|                 0.88911|0.64169|   0.23351|
|         Netherlands|      

In [21]:
import numpy as np

In [22]:
datardd2015 = data2015.rdd.map(lambda x: np.array([x[1],x[2], x[3], x[4], x[5]]))
datardd2016 = data2016.rdd.map(lambda x: np.array([x[1],x[2], x[3], x[4], x[5]]))
datardd2017 = data2017.rdd.map(lambda x: np.array([x[-3],x[-2], x[-1], x[4], x[5]]))
datardd2018 = data2018.rdd.map(lambda x: np.array([x[1],x[2], x[3], x[4], x[5]]))
datardd2019 = data2019.rdd.map(lambda x: np.array([x[1],x[2], x[3], x[4], x[5]]))

In [23]:
rdd = datardd2015.union(datardd2016)
rdd = rdd.union(datardd2017)
rdd = rdd.union(datardd2018)
rdd = rdd.union(datardd2019)

(6) n (= 조사 데이터의 개수)을 구하시오. Action 사용. (5점)

In [24]:
rdd.count()

782

In [130]:
rdd_kv = rdd.map(lambda x:(x[0], (x[1], x[2], x[3], x[4])))

In [156]:
rdd_x = rdd_kv.map(lambda x: np.array(x[1]))

(7) rdd의 튜플의 평균을 계산하시오. (10점)

In [25]:
rdd.mean()

array([5.3790179 , 0.91604748, 0.61241558, 0.41109083, 0.21857584])

(8) rdd의 표준편차를 계산하시오. (10점)

In [26]:
rdd.stdev()

array([1.12673535, 0.4070796 , 0.24814983, 0.15278264, 0.12224251])

(9) map과 reduce를 이용해 피어슨 상관관계 (4개)를 numpy array로 한꺼번에 계산하시오. (10점)

힌트: map으로 numpy array x의 $(x - \mu_x) / \sigma_x$를 계산한 다음, 다시 map으로 <code>[ x[0] * t for in arr[1:] ]</code> 를 계산해 numpy arrray로 만들어준 다음 sum을 구하면 피어슨 상관관계식의 시그마 식 부분을 구할 수 있다.